In [1]:
import numpy as np
import cifar10
import matplotlib.pyplot as plt

In [2]:
from keras.models import Sequential
from keras.layers import Activation, Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from keras.wrappers.scikit_learn import KerasClassifier

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix

In [4]:
cifar10.data_path = "data/CIFAR-10/"

In [5]:
cifar10.maybe_download_and_extract()

Data has apparently already been downloaded and unpacked.


In [6]:
class_names = cifar10.load_class_names()
class_names

Loading data: data/CIFAR-10/cifar-10-batches-py/batches.meta


['airplane',
 'automobile',
 'bird',
 'cat',
 'deer',
 'dog',
 'frog',
 'horse',
 'ship',
 'truck']

In [7]:
images_train, cls_train, labels_train = cifar10.load_training_data()
images_test, cls_test, labels_test = cifar10.load_test_data()

Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_1
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_2
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_3
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_4
Loading data: data/CIFAR-10/cifar-10-batches-py/data_batch_5
Loading data: data/CIFAR-10/cifar-10-batches-py/test_batch


In [8]:
images_train.shape, cls_train.shape, labels_train.shape

((50000, 32, 32, 3), (50000,), (50000, 10))

In [9]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\ensemble\weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [10]:
#X_train, X_cross_val, Y_train, Y_cross_val = train_test_split(images_train, labels_train, random_state=0, test_size=0.2)

In [11]:
"""
Build Convolutional Neural Network
"""

'\nBuild Convolutional Neural Network\n'

In [20]:
def build_clf():
    
    clf = Sequential()
    
    units = 64
    conv_dropout_rate = 0.25
    dense_dropout_rate = 0.25
    conv_kernal_size = (5, 5)
    
    clf.add(Conv2D(filters=units, kernel_size = conv_kernal_size, padding="same", input_shape=(32, 32, 3), activation='relu', data_format="channels_last"))
    clf.add(MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='same', data_format='channels_last'))
    clf.add(Dropout(rate=conv_dropout_rate))
    
    clf.add(Conv2D(filters=units, kernel_size = conv_kernal_size, padding="same", activation='relu', data_format="channels_last"))
    clf.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1), padding='same', data_format='channels_last'))
    clf.add(Dropout(rate=conv_dropout_rate))

    clf.add(Conv2D(filters=units, kernel_size = conv_kernal_size, padding="same", activation='relu', data_format="channels_last"))
    clf.add(MaxPooling2D(pool_size=(2,2), strides=(1, 1), padding='same', data_format='channels_last'))
    clf.add(Dropout(rate=conv_dropout_rate))

    
    clf.add(Flatten())

    clf.add(Dense(units=units, activation='relu'))
    clf.add(Dropout(rate=dense_dropout_rate))
    clf.add(Activation(activation='relu'))
    
    
    clf.add(Dense(units=units, activation='relu'))
    clf.add(Dropout(rate=dense_dropout_rate))
    clf.add(Activation(activation='relu'))
    
    clf.add(Dense(units=units, activation='relu'))
    clf.add(Dropout(rate=dense_dropout_rate))
    clf.add(Activation(activation='relu'))
    
    clf.add(Dense(units=len(class_names), activation='softmax'))
    
    clf.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
    return clf

In [21]:
clf = KerasClassifier(build_fn=build_clf)
#clf_arr = [KerasClassifier(build_fn=build_clf) for i in range(5)]

In [22]:
clf.fit(images_train, labels_train, epochs=1, batch_size=100, shuffle=True)

Epoch 1/1
50000/50000 [==============================] - 348s 7ms/step - loss: 2.2416 - acc: 0.1430


In [ ]:
"""for clf in clf_arr :
    X_train, X_cross_val, Y_train, Y_cross_val = train_test_split(images_train, labels_train, random_state=clf_arr.index(clf), test_size=0.2)
    clf.fit(X_train, Y_train, validation_data = (X_cross_val, Y_cross_val),epochs=20, batch_size=500, verbose=1)
    #print("Train Score", clf.score(X_train, Y_train))
    #print("Cross Val Score", clf.score(X_cross_val, Y_cross_val))
del X_train, X_cross_val, Y_train, Y_cross_val"""

In [ ]:
"""for i in range(10):
    clf.fit(images_train, labels_train, batch_size=100, epochs=1)
    Y_train_pred = clf.predict(images_train)
    Y_test_pred = clf.predict(images_test)
    Y_train_pred = [np.argmax(y) for y in Y_train_pred]
    Y_test_pred = [np.argmax(y) for y in Y_test_pred]
    train_accuracy_score = accuracy_score(cls_train, Y_train_pred)
    print("Train Accuracy after", i+1,"iterations:", train_accuracy_score)
    test_accuracy_score = accuracy_score(cls_test, Y_test_pred)
    print("Test Accuracy after", i+1,"iterations:", test_accuracy_score)
    if train_accuracy_score > 0.9 :
        break"""

In [15]:
from sklearn.metrics import accuracy_score

In [23]:
Y_train_pred = clf.predict(images_train)
Y_test_pred = clf.predict(images_test)

In [24]:
Y_train_pred.shape, Y_test_pred.shape

((50000,), (10000,))

In [25]:
Y_train_pred = [np.argmax(y) for y in Y_train_pred]
Y_test_pred = [np.argmax(y) for y in Y_test_pred]

In [26]:
#print(confusion_matrix(Y_train_pred, Y_train))
print(classification_report(Y_train_pred, cls_train))
#print(confusion_matrix(Y_test_pred, Y_test))
print(classification_report(Y_test_pred, cls_test))

"""

 0.62      0.57      0.59     50000
 0.61      0.56      0.58     10000
 
 0.59      0.57      0.57     50000
 0.59      0.56      0.56     10000
 
"""


             precision    recall  f1-score   support

          0       1.00      0.10      0.18     50000
          1       0.00      0.00      0.00         0
          2       0.00      0.00      0.00         0
          3       0.00      0.00      0.00         0
          4       0.00      0.00      0.00         0
          5       0.00      0.00      0.00         0
          6       0.00      0.00      0.00         0
          7       0.00      0.00      0.00         0
          8       0.00      0.00      0.00         0
          9       0.00      0.00      0.00         0

avg / total       1.00      0.10      0.18     50000

             precision    recall  f1-score   support

          0       1.00      0.10      0.18     10000
          1       0.00      0.00      0.00         0
          2       0.00      0.00      0.00         0
          3       0.00      0.00      0.00         0
          4       0.00      0.00      0.00         0
          5       0.00      0.00      0.00

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


'\n\n 0.62      0.57      0.59     50000\n 0.61      0.56      0.58     10000\n \n 0.59      0.57      0.57     50000\n 0.59      0.56      0.56     10000\n \n'

In [ ]:
predictions = [class_names[i] for i in Y_test.astype(int)]
np.savetxt("preds.csv", predictions, fmt = "%s")